In [27]:
import os, json
import pandas as pd
import random
import numpy as np
from collections import defaultdict, Counter


In [28]:
data = pd.read_json(
    "/Users/guida/mfc_fine_tuning/fine_tuned_roberta/final_model_no_15/inferences/kl_divergence_15/samples_for_annotation/sample_sfu_combined_stratified.json",
    lines=True
)
print(len(data))
duplicates = data[data.duplicated('sentence_text', keep=False)]
duplicates

560


,sentence_text,label


## Prepare new files

In [29]:
random.seed(42)

In [30]:
df = data.sample(frac=1, random_state=42).reset_index(drop=True)

overlap_sentences = df.iloc[:60]
unique_sentences = df.iloc[60:]

for i in range(1, 3):  # i = 1, 2  -> sessions 2 and 3
    overlap = overlap_sentences.iloc[i*20:(i+1)*20]
    unique = unique_sentences.iloc[i*180:(i+1)*180]
    session = pd.concat([overlap, unique]).sample(frac=1, random_state=42+i)
    session.to_json(f'annotation_session_{i+1}.jsonl', orient='records', lines=True)

sessions = []
for i in range(1, 3):  # again load sessions 2 and 3 only
    s = pd.read_json(f'annotation_session_{i+1}.jsonl', lines=True)
    sessions.append(s)

for i, s in enumerate(sessions, 2):  # start enumeration from 2 to reflect session numbers
    total = len(s)
    overlap = len(s.merge(overlap_sentences, on='sentence_text'))
    unique = total - overlap
    dupes_within = s.duplicated('sentence_text').sum()
    print(f'Session {i}: total={total}, overlap={overlap}, unique={unique}, duplicates_within={dupes_within}')

if len(sessions) == 2:
    overlap_across = sessions[0].merge(sessions[1], on='sentence_text')
    print(f'Total distinct overlapping sentences across sessions 2 and 3: {len(overlap_across)}')

Session 2: total=200, overlap=20, unique=180, duplicates_within=0
Session 3: total=160, overlap=20, unique=140, duplicates_within=0
Total distinct overlapping sentences across sessions 2 and 3: 0


In [31]:
sesh2 = pd.read_json('annotation_session_3.jsonl', lines=True)
sesh2['sentence_text'].value_counts()

sentence_text
I would hope Canadians in Quebec and the ROC will send him a message he can't ignore, boycott anything he touches.                                                                                                                                                                                                                               1
While addiction may call for forgiveness if I don't care for my children, I still must do so or arrange for others to do so.                                                                                                                                                                                                                     1
No kidding G+M .....are you just waking up from a 6 year slumber to that which rest of us with a brain in our head knew since 2006???                                                                                                                                                                               

In [ ]:
session_paths = [
    "/Users/guida/matteo_annotation_app/templates/data/annotation_session_1.jsonl",
    "/Users/guida/matteo_annotation_app/templates/data/annotation_session_2.jsonl",
    "/Users/guida/matteo_annotation_app/templates/data/annotation_session_3.jsonl"
]

all_sessions = pd.concat([pd.read_json(path, lines=True) for path in session_paths], ignore_index=True)

label_counts = all_sessions['label'].value_counts()
label_percentages = all_sessions['label'].value_counts(normalize=True) * 100

label_distribution = pd.DataFrame({
    'count': label_counts,
    'percentage': label_percentages.round(2)
})
label_distribution

,count,percentage
label,,
1,102,18.21
6,92,16.43
5,76,13.57
9,60,10.71
11,51,9.11
8,44,7.86
4,44,7.86
12,41,7.32
3,38,6.79
